In [1]:
import pandas as pd
import numpy as np

In [2]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('notebook_repr_html',False)

In [6]:
dbname = 'opensyl'
username = 'parama' 

In [4]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url

postgres://parama@localhost/opensyl


In [7]:
# connect:
try:
    con = psycopg2.connect(database = 'opensyl', user = 'parama' , password = 'pargop')
except:
    print 'Unable to connect!'

In [8]:
# First QUERY to select the authornames to be classified

sql_query = """
SELECT text.id, text.authors text.surname FROM text
INNER JOIN citation ON citation.text_id = text.id
WHERE text.valid = true AND
text.display = true
GROUP BY text.id
"""
frm_sql = pd.read_sql_query(sql_query,con)

In [15]:
frm_sql.shape

(933635, 2)

In [ ]:
df = frm_sql

In [ ]:
# Remove special characters
df.authors = df.authors.str.replace('[^a-zA-Z ]', '') 

# Remove surname 
sn = list(df.surname)
fn = list(df.authors)
nn = [fullname.replace(surname,'') for surname, fullname in zip(sn,fn)]
df['authors'] = nn

#Retain longest word (needs repairing)
fn = list(df.authors)
res = []
for el in fn:
    words = el.split()
    if words:
        longestword = max(words, key=len)
        res.append(longestword)
    else:
        res.append('')


df['authors'] = res
df.to_csv('sql_authors.csv',index=False)

In [ ]:
# Here : stopwords from names are removed and the classifier applied on to them - then stored in database

non_names = ['editor','author','Admin','Administration','Administrative','General','Publisher','Publishing',
             'Random','Lawyer','Engineer','President','Author','Editor','writer','Writer','Reader','College',
            'School','University','Institute','Institution','Government','Authority','Member','Authorities']
genders = []
for everyname in df.authors:
    if len(everyname) < 2 or (everyname in non_names):
        newgender = 'Unknown' 
    else:
        mprob = clf.predict_proba(nametogender(everyname))
        if mprob[0,0] > 0.60:
            newgender = 'm'
        elif mprob[0,1] > 0.60:
            newgender = 'f'
        else:
            newgender = 'Unknown'         
    genders.append(newgender)
df['gender'] = genders

df.to_sql('gender_info_prob', engine)

In [19]:
# This query gives erroneous results I think

sql_query = """
SELECT gender_info_prob.id AS t_id,institution.name AS inst,institution.state INTO inst_table1 FROM 
gender_info_prob INNER JOIN citation ON gender_info_prob.id = citation.text_id 
INNER JOIN institution_document ON citation.document_id = institution_document.document_id
INNER JOIN institution ON institution_document.institution_id = institution.id  
WHERE gender != 'Unknown'
"""
frm_sql = pd.read_sql_query(sql_query,con)

TypeError: 'NoneType' object is not iterable

In [20]:
frm_sql.shape

(2646909, 3)

In [21]:
frm_sql.head()

      t_id                               inst state
0  4524953              Iowa State University    IA
1  4525341                 Harvard University    MA
2  4525341                 Harvard University    MA
3  3620539  The University of Texas at Dallas    TX
4  1811011                 Harvard University    MA